In [1]:
!git clone https://github.com/facebookresearch/ImageBind
# cd ImageBind

Cloning into 'ImageBind'...
remote: Enumerating objects: 117, done.
remote: Counting objects: 100% (70/70), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 117 (delta 48), reused 35 (delta 35), pack-reused 47
Receiving objects: 100% (117/117), 2.64 MiB | 15.27 MiB/s, done.
Resolving deltas: 100% (53/53), done.


In [2]:
cd ImageBind

/kaggle/working/ImageBind


In [ ]:

!pip install -r requirements.txt
!pip install soundfile

  Cloning https://github.com/facebookresearch/pytorchvideo.git (to revision 28fe037d212663c6a24f373b94cc5d478c8c1a1d) to /tmp/pip-install-vxq57crw/pytorchvideo_95d194c5841d455e8545a01694605174
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/pytorchvideo.git /tmp/pip-install-vxq57crw/pytorchvideo_95d194c5841d455e8545a01694605174
  Running command git rev-parse -q --verify 'sha^28fe037d212663c6a24f373b94cc5d478c8c1a1d'
  Running command git fetch -q https://github.com/facebookresearch/pytorchvideo.git 28fe037d212663c6a24f373b94cc5d478c8c1a1d
  Running command git checkout -q 28fe037d212663c6a24f373b94cc5d478c8c1a1d
  Resolved https://github.com/facebookresearch/pytorchvideo.git to commit 28fe037d212663c6a24f373b94cc5d478c8c1a1d
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 

In [ ]:
!pip install qdrant-client>=1.1.1
!pip install -U sentence-transformers


In [ ]:
%pip install opendatasets gradio qdrant-client transformers sentence_transformers sentencepiece tqdm

In [ ]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/sriramr/apples-bananas-oranges")


In [ ]:
from qdrant_client import models, QdrantClient
import random
from PIL import Image
import os
from tqdm import tqdm
import torch
import sys
sys.path.append("./ImageBind/")
import imagebind
from imagebind.models import imagebind_model
from imagebind.models.imagebind_model import ModalityType

from imagebind import data
import uuid


In [ ]:
# Initialize Qdrant client
qdrant = QdrantClient(":memory:")

def get_image_paths(directory):
    """
    Retrieve paths of images from the specified directory.

    Args:
    directory (str): The directory containing the images.

    Returns:
    list: List of image paths.
    """
    image_paths = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
                image_path = os.path.join(root, file)
                image_paths.append(image_path)
    return image_paths

# Directory paths
apple_directory = './apples-bananas-oranges/original_data_set/freshapples/'
banana_directory = './apples-bananas-oranges/original_data_set/freshbanana'
orange_directory = './apples-bananas-oranges/original_data_set/freshoranges'

# Get image paths for different categories
image_paths_apple = get_image_paths(apple_directory)
image_paths_banana = get_image_paths(banana_directory)
image_paths_orange = get_image_paths(orange_directory)

all_image_paths = [image_paths_orange, image_paths_banana, image_paths_apple]


In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

model = imagebind_model.imagebind_huge(pretrained=True)
model.eval()
model.to(device)

embeddings_list = []
for image_paths in [image_paths_orange, image_paths_banana, image_paths_apple]:
  inputs = {ModalityType.VISION: data.load_and_transform_vision_data(image_paths, device)}
  # print(inputs)
  with torch.no_grad():
    embeddings = model(inputs)
    embeddings_list.append(embeddings)

In [ ]:
import uuid
# Initialize Qdrant client and load collection
client = QdrantClient(":memory:")
client.recreate_collection(collection_name = "imagebind_data",
vectors_config = {"image": models.VectorParams( size = 1024, distance = models.Distance.COSINE ) } )
points = []
# Iterate over each embeddings and corresponding image paths
for idx, (embedding, image_paths) in enumerate(zip(embeddings, all_image_paths)):
  # print(idx,embeddings['vision'],image_paths)
  for sub_idx, sample in enumerate(image_paths):
    # print(image_paths)
  # Convert the sample to a dictionary
    payload = {"path": sample}
    # print(embedding)
    # Generate a unique UUID for each point
    point_id = str(uuid.uuid4())
    points.append(models.PointStruct(id=point_id,
    vector= {"image": embeddings['vision'][sub_idx]},
    payload=payload)
    )
client.upsert(collection_name="imagebind_data", points=points)

In [ ]:
!pip install opencv-python ultralytics


In [ ]:
import cv2
from ultralytics import YOLO

yolo_model = YOLO("yolov9c.pt")

In [ ]:
def predict(chosen_model, img, classes=[], conf=0.5):
    """
    Perform object detection on the input image.

    Args:
    chosen_model: YOLO model instance.
    img: Input image.
    classes: List of classes to detect (optional).
    conf: Confidence threshold for detections (optional).

    Returns:
    results: Detection results.
    """
    if classes:
        results = chosen_model.predict(img, classes=classes, conf=conf)
    else:
        results = chosen_model.predict(img, conf=conf)

    return results

def predict_and_detect(chosen_model, img, classes=[], conf=0.5, rectangle_thickness=2, text_thickness=1):
    """
    Perform object detection on the input image and save cropped images of detected objects.

    Args:
    chosen_model: YOLO model instance.
    img: Input image.
    classes: List of classes to detect (optional).
    conf: Confidence threshold for detections (optional).
    rectangle_thickness: Thickness of rectangle around detected objects (optional).
    text_thickness: Thickness of text overlay on detected objects (optional).

    Returns:
    img: Image with detections overlayed.
    results: Detection results.
    """
    results = predict(chosen_model, img, classes, conf=conf)
    for result in results:
        for box in result.boxes:
            cropped_object = img[int(box.xyxy[0][1]):int(box.xyxy[0][3]), int(box.xyxy[0][0]):int(box.xyxy[0][2])]
            cv2.imwrite(f"{result.names[int(box.cls[0])]}"+".jpg", cropped_object)

    return img, results

In [ ]:
image = cv2.imread("/kaggle/working/ImageBind/apples-bananas-oranges/original_data_set/rottenoranges/Screen Shot 2018-06-12 at 11.26.07 PM.png")
result_img, _ = predict_and_detect(yolo_model, image, classes=[], conf=0.3)


In [ ]:
import gradio as gr
import gradio
import cv2
import base64
import numpy as np
from PIL import Image
from qdrant_client import models, QdrantClient
from ultralytics import YOLO
import torch

# Initialize Qdrant client
# client = QdrantClient(":memory:")

# Initialize YOLO model
# yolo_model = YOLO("yolov5s.pt")

# Function to perform object detection on the input image
def detect_objects(image):
    # Convert image to numpy array
    img_array = np.array(image)
    # Perform object detection using YOLO
    results = yolo_model.predict(img_array)
    # Convert the results to PIL format
    result_images = [Image.fromarray(obj) for obj in results.imgs]
    # Return the detected objects
    return result_images

# Function to search for similar objects in Qdrant database
def search_similar_objects(detected_objects):
    similar_images = []
    for obj in detected_objects:
        # Convert the detected object image to base64
        _, img_encoded = cv2.imencode('.jpg', cv2.cvtColor(np.array(obj), cv2.COLOR_RGB2BGR))
        img_base64 = base64.b64encode(img_encoded).decode('utf-8')
        
        # Convert the base64 image to vector format
        obj_vector = image_to_vector(obj)
        
        # Search for similar objects in the Qdrant database
        object_hits = client.search(
            collection_name='imagebind_data',
            query_vector=models.NamedVector(
                name="object",
                vector=obj_vector.tolist()
            )
        )
        
        # Add the search results to the list of similar images
        if object_hits:
            for hit in object_hits:
                similar_images.append(hit.payload['image_base64'])
    
    # Return the list of similar images
    return similar_images

# Function to convert image to vector using ImageBind model
def image_to_vector(image):
    # Load and transform the image data
    image_data = {"image": data.load_and_transform_vision_data([image], device)}
    
    # Generate embeddings for the image
    with torch.no_grad():
        embeddings = model(image_data)
    
    # Extract the vision embeddings
    vision_embeddings = embeddings['vision']
    
    # Return the embeddings as a vector
    return vision_embeddings[0].tolist()

# Gradio Interface
iface = gr.Interface(
    fn=detect_objects,
    inputs=[gr.Image(label="image_query", type="filepath")],
    outputs=[
        gr.Image(label="Image")],
    title="Object Detection and Similar Object Search",
    description="Upload an image to detect objects and find similar objects using ImageBind and Qdrant.",
)

iface.launch()

In [ ]:
!pip install -U gradio
